In [1]:
import os 
import sys
from sympy import *
import pandas as pd
from scipy.integrate import odeint
from sympy import symbols, init_printing
import matplotlib.pyplot as plt
import cmath
import numpy as np

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from src.IO.xcel import *
from src.core.wrapper import *
from src.IO.get_file_names import *
from src.utils.equ_comp import *
from numpy.linalg import inv
from src.utils.get_linearization_equ import *


### Define the case file for the AC/DC system under consideration:
The case folder for example "Case_14bus" includes the following user-defined files:
- **ieee14AC.xlsx**: this file includes the original AC system with their actual loads. This file conforms the requirement of ANDES case file, and it is only for power flow solution purposes. Please make sure the 'u' column of the 'PQ','PV' and 'Slack' sheet are correctly assigned. '0': not connected and '1': connected. Also, ensure the 'Bus', Line' and 'Shunt' sheet as per the requirement. 
- **mtdc_net_14AC.xlsx**: this file includes the DC system description.
    - **Bus**: details of DC bus parameters and active power injcetions. please ensure the appropriate sign (+ve or -ve) for 'Pinj' column of 'Bus' sheet to reflect the direction of 
net injection. 
    - **DC_params_status**: default description of the type of dc-side parameters [c: bus capacitances, kp, ki: dc slack bus PI gains, Vref: dc slack bus reference voltage]
    - **Line**: details of dc lines and parameters.
    - **Slack**: dc slack bus no. and the ac bus no. where dc slack bus is connected.
    - **Interface**: DC and AC bus connection mapping. The numbering of DC and AC network are done independently. 
- **ieee14AC_ss.xlsx**: this file follows the standard architecture of **ieee14AC.xlsx** with modifications for dynamic componenets e.g. SG, Governor, Exciter, GFM and GFL. The parameters of SG, Governor, Exciter are mentioned in 'GEN_params_status' sheet, while for GFM and GFL are mentioned in 'GFM_params_status' and 'GFL_params_status', respectively. 

The AC and DC parameters can be defined as:
- 'sym': Symbolic - parameters are treated as symbolic variables for analytical computations or symbolic modeling.
- 'num': Numeric - parameters are assigned specific numeric values for direct numerical simulations.
- 'user': User-defined - parameters are provided by the user as custom inputs for tailored simulations or system-specific settings.

Note that, **ieee14AC_with_dc_mod.xlsx**, **pf_info_Case_14bus.xlsx**, **pf_info_extendedCase_14bus.xlsx** will be created in the design procedure.


In [2]:
# Define base path relative to the project directory
case_path = os.path.join(parent_dir, "cases\\Case_14bus")

# update the default setting of the parameters (if needed)

# (1) 'sym': Symbolic - parameters are treated as symbolic variables for analytical computations or symbolic modeling.
# (2) 'num': Numeric - parameters are assigned specific numeric values for direct numerical simulations.
# (3) 'user': User-defined - parameters are provided by the user as custom inputs for tailored simulations or system-specific settings.

# SG params: M	D	xd	xq	xd1	xq1	Td10	Tq10	ra	Tm	Rgov	Te	Ke	Vref
user_updates_gen_params = {
    "M": 'num',   
    "D": 'num'
}
# GFL params: KpPLL	KiPLL	kdroop	xq	ra	kxl	Kp	Ki	Sl
user_updates_gfl_params = {
    "xq": 'sym',   
    "ra": 'sym'
}
# DC params: c	kp	ki	Vref
user_updates_dc_params = {
    "c": 'sym'
}

### File names:
- Import all the necessary file names (both AC/DC)

In [3]:
dc_case_file,ac_base_file,ac_pf,\
    ac_base_sym, eq_data_file, eq_data_file_extended = case_files(case_path)

User provided Case Name: Case_14bus


### DC functions for DC network:
- Call DC wrapper to initialize, solve DC power flow, and create DC symbolic equations

In [4]:
dc_system,dc_network,dc_syms = dc_wrapper(dc_case_file,user_updates_dc_params)

Default status DC-net params      c   kp   ki Vref
0  num  num  num  num
Current status DC-net params      c   kp   ki Vref
0  sym  num  num  num
DC bus Voltages (p.u.): 
 [1.00231504 1.00331671 1.00257424 1.00158247 1.00129059 1.00031567
 1.00306748 1.00232482 1.00058305 1.        ]
DC bus Powers (1e3 MW): 
 [ 0.1         0.2         0.05        0.03        0.1        -0.2
 -0.025      -0.025      -0.1        -0.12905903]


### DAE Formation:
- Create the coupled DAE equations for AC and DC network

In [5]:
system,bus,model,alg = ac_wrapper(ac_base_sym,user_updates_gen_params,user_updates_gfl_params)

f,g,x,y = ac_dc_connection(system,bus,model,alg,dc_system,dc_network,dc_syms)

Default status of SG params:      M    D   xd   xq  xd1  xq1 Td10 Tq10   ra   Tm Rgov   Te   Ke Vref
0  sym  sym  sym  sym  sym  sym  sym  sym  sym  sym  sym  sym  sym  sym
Default status of GFL params:   KpPLL KiPLL kdroop   xq   ra  kxl   Kp   Ki    Sl
0  user  user   user  num  num  num  num  num  user
Default status of GFM params: Empty DataFrame
Columns: [mp, mq, gamma, kpv, kiv, tau, xq, ra, Sl]
Index: []
Current status of SG params:      M    D   xd   xq  xd1  xq1 Td10 Tq10   ra   Tm Rgov   Te   Ke Vref
0  num  num  sym  sym  sym  sym  sym  sym  sym  sym  sym  sym  sym  sym
Current status of GFL params:   KpPLL KiPLL kdroop   xq   ra  kxl   Kp   Ki    Sl
0  user  user   user  sym  sym  num  num  num  user
Current status of GFM params: Empty DataFrame
Columns: [mp, mq, gamma, kpv, kiv, tau, xq, ra, Sl]
Index: []


### Update AC side load data:
- Based on the updated DC injections update the **PQ** sheet of the **ieee14AC.xlsx**, and create a new file **ieee14AC_with_dc_mod.xlsx**

In [6]:
ac_pf_update(ac_base_file,ac_pf,'PQ',dc_system,dc_network,system)

c:\Users\hoss258\Projects_2324\ecomp_code_publish\ecomp-stability\SyNAPS\cases\Case_14bus\ieee14AC_with_dc_mod.xlsx already exists.


### AC power flow, bus-wise injection computation for AC-side extended network and equilibrium value assignment:
- Solve AC power flow with the **ieee14AC_with_dc_mod.xlsx**, we used ANDES pf solver for this step. The power flow results are saved in **pf_info_Case_14bus.xlsx** file.
- The AC-side buses are extended one-hop to include the internal node of the generating resources. 
- The extended network results are saved in **pf_info_extendedCase_14bus.xlsx** file. 

In [7]:
base_equ_comp(case_path,ac_pf,eq_data_file,system,alg)

ac_ext_net_sol = extended_equ_comp(eq_data_file,eq_data_file_extended,system,alg)

equ_value = equ_val_assignment(dc_network,dc_system,dc_syms,ac_ext_net_sol,system,bus,model,alg)

Working directory: "c:\Users\hoss258\Projects_2324\ecomp_code_publish\ecomp-stability\SyNAPS\examples"
> Loaded generated Python code in "C:\Users\hoss258\.andes\pycode".
Parsing input file "c:\Users\hoss258\Projects_2324\ecomp_code_publish\ecomp-stability\SyNAPS\cases\Case_14bus\ieee14AC_with_dc_mod.xlsx"...
GENCLS: unused data {'xd': 1.8, 'xq': 1.75, 'xd2': 0.23, 'xq1': 0.8, 'xq2': 0.23, 'Td10': 6.5, 'Td20': 0.06, 'Tq10': 0.2, 'Tq20': 0.05}
GENCLS: unused data {'xd': 1.8, 'xq': 1.75, 'xd2': 0.28, 'xq1': 0.8, 'xq2': 0.28, 'Td10': 6.5, 'Td20': 0.06, 'Tq10': 0.2, 'Tq20': 0.05}
GENCLS: unused data {'xd': 1.8, 'xq': 1.75, 'xd2': 0.34, 'xq1': 0.8, 'xq2': 0.34, 'Td10': 6.5, 'Td20': 0.06, 'Tq10': 0.2, 'Tq20': 0.05}
GENCLS: unused data {'xd': 1.8, 'xq': 1.75, 'xd2': 0.28, 'xq1': 0.8, 'xq2': 0.28, 'Td10': 6.5, 'Td20': 0.06, 'Tq10': 0.2, 'Tq20': 0.05}
GENCLS: unused data {'xd': 1.8, 'xq': 1.75, 'xd2': 0.34, 'xq1': 0.8, 'xq2': 0.34, 'Td10': 6.5, 'Td20': 0.06, 'Tq10': 0.2, 'Tq20': 0.05}
Input fil

-> Single process finished in 0.5867 seconds.
Folder already exists: c:\Users\hoss258\Projects_2324\ecomp_code_publish\ecomp-stability\SyNAPS\cases\Case_14bus\andes_report
Moved file: ieee14AC_with_dc_mod_out.txt -> c:\Users\hoss258\Projects_2324\ecomp_code_publish\ecomp-stability\SyNAPS\cases\Case_14bus\andes_report\ieee14AC_with_dc_mod_out.txt


### Verfication of the equilibrium point and DAEs:
- Replace the equilibrium values in $f$ and $g$ equations and check whether the equilibrium is correct or not. 
- Note that the equations are derived from symbolic-numeric tool, while the equilibrium values are based on ANDES pf solutions 

In [8]:
if max(abs(g.subs(equ_value))) < 1e-5 and max(abs(f.subs(equ_value))) < 1e-5:
    print("Correct Equilibrium")
else:
    print("Incorrect Equilibrium")

Correct Equilibrium


### Jacobians:
- Compute required jacobians for small-signal model derivation

In [9]:
# compute jacobians
fx_acdc = f.jacobian(x)
fy_acdc = f.jacobian(y)
gx_acdc = g.jacobian(x)
gy_acdc = g.jacobian(y)

fx_acdc.simplify()
fy_acdc.simplify()
gx_acdc.simplify()
gy_acdc.simplify()

### Evaluate the Jacobians and compute A matrix
- Derivation: $\dot{x} = f(x,y)$, $0 = g(x,y)$ $\Rightarrow$ $\Delta \dot{x} = (f_x - f_yg_y^{-1}g_x)\Delta x$ $\Rightarrow$ $\Delta \dot{x} = A\Delta x$

In [10]:
fx_sub = fx_acdc.subs(equ_value)
fy_sub = fy_acdc.subs(equ_value)
gx_sub = gx_acdc.subs(equ_value)
gy_sub = gy_acdc.subs(equ_value)

inv_gy = inv(np.array(gy_sub,dtype=float))
if np.linalg.det(inv_gy) != 0:
    print("Correct Gy")

A = fx_sub - fy_sub@inv_gy@gx_sub

Correct Gy


### Display parametric A matrix
- With the current setting the derived A matrix contains GFL size, droop, pll gains as parameters

In [11]:
A

⎡           0                          0                          0            ↪
⎢                                                                              ↪
⎢           0                          0                          0            ↪
⎢                                                                              ↪
⎢           0                          0                          0            ↪
⎢                                                                              ↪
⎢           0                          0                          0            ↪
⎢                                                                              ↪
⎢           0                          0                          0            ↪
⎢                                                                              ↪
⎢   -0.211151573162178        0.0566789954830791         0.0346281979201073    ↪
⎢                                                                              ↪
⎢   0.0388477801666011      

### Eigenvalues of A matrix for a given setting:
- For some give values of the user-defined parameters the eigen values are computed.
-  Check stability by observing the eigenvalues.

In [12]:
# update some parameter values with user-defined ones
mp_hat = 0.010
values_ud = {}
for k in range(len(system.gfl)):
    values_ud[model.gfl_cap[k]] = 1.0
    values_ud[model.k_droop[k]] = 1/mp_hat 
    values_ud[model.kiPLL[k]] = 1.7 #5  
    values_ud[model.kpPLL[k]] = 0.1 #0.2 


A_matrix1 = A.subs(values_ud)
A1 = np.array(A_matrix1,dtype=float)


eigenvalues,_ = np.linalg.eig(A1) 

print('Eigenvalues:')
for k in range(A_matrix1.shape[0]):
    print(np.round(eigenvalues[k],4))

Eigenvalues:
(-60.7665+0j)
(-53.074+0j)
(2.2322+26.7806j)
(2.2322-26.7806j)
(-23.5724+5.7962j)
(-23.5724-5.7962j)
(-22.9512+6.1812j)
(-22.9512-6.1812j)
(-19.8281+0j)
(-12.4201+0j)
(-10.5029+0j)
0j
(-0.5534+8.6686j)
(-0.5534-8.6686j)
(-1.0218+6.3393j)
(-1.0218-6.3393j)
(-0.2199+6.6974j)
(-0.2199-6.6974j)
(-0.1827+7.6491j)
(-0.1827-7.6491j)
(-4.876+0j)
(-3.956+0j)
(-2.9046+0j)
(-2.2028+0j)
(-1.995+0j)
(-0.0473+0j)
(-0.1023+0.177j)
(-0.1023-0.177j)
(-0.4305+0j)
(-0.7385+0j)
(-0.6457+0j)
